<a href="https://colab.research.google.com/github/ftorres-10cero/Google-Colab-IA-Pix2Pix/blob/dev/IA_Notebook_Pix2Pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tutorial Dot CSV Pix2Pix con TensorFlow 2.0
# https://www.youtube.com/watch?v=YsrMGcgfETY&t=1572s

In [0]:
# Activamos la versión de tensorflow que vamos a usar: 2.0
%tensorflow_version 2.x

In [0]:
# Realizamos las importaciones de librerias y declaramos las variables a usar

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


PATH = "/content/drive/My Drive/pix2pix"

# Ruta de datos de entrada
INPATH = PATH + '/in'
# Ruta de datos de salida
OUTPATH = PATH + '/out'
# RUTA de los checkpoints
CKPATTH = PATH + '/checkpoints'

imgurls = !ls -1 "{INPATH}"

n = 500
train_n = round(n * 0.80)

# Listado randomizado
randurls = np.copy(imgurls)

np.random.seed(23)
np.random.shuffle(randurls)

# Partición train/test

tr_urls = randurls[:train_n]
ts_urls = randurls[train_n:n]

## print(len(imgurls), len(tr_urls), len(ts_urls))

In [0]:
# Declaramos las constantes que vamos a usar y definimos las funciones básicas

IMG_WIDTH = 256
IMG_HEIGHT = 256

# Reescalar imágenes
def resize(inimg, tgimg, height, width):

  inimg = tf.image.resize(inimg, [height, width])
  tgimg = tf.image.resize(tgimg, [height, width])

  return inimg, tgimg

# Normaliza el rango [-1, +1] la imagen
def normalize(inimg, tgimg):
  inimg = (inimg / 127.5) - 1
  tgimg = (tgimg / 127.5) - 1

  return inimg, tgimg

# Aumentacion de datos : Rasdom Crop + Flip
def random_jitter(inimg, tgimg):

  inimg, tgimg = resize(inimg, tgimg, 286, 286)

  stacked_image = tf.stack([inimg, tgimg], axis=0)
  cropped_image = tf.image.random_crop(stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH,3])

  inimg, tgimg = cropped_image[0], cropped_image[1]

  if tf.random.uniform(()) > 0.5:

    inimg = tf.image.flip_left_right(inimg)
    tgimg = tf.image.flip_left_right(tgimg)

  return inimg, tgimg  

# Carga de imagenes
def load_image(filename, augment=False):

  inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(INPATH + '/' + filename)), tf.float32)[..., :3]
  tgimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(OUTPATH + '/' + filename)), tf.float32)[..., :3]  

  inimg, tgimg = resize(inimg, tgimg, IMG_HEIGHT, IMG_WIDTH)

  if augment:
    inimg, tgimg = random_jitter(inimg, tgimg)

  inimg, tgimg = normalize(inimg, tgimg)

  return inimg, tgimg


def load_train_image(filename):
  return load_image(filename, True)

def load_test_image(filename):
  return load_image(filename, False)

# Vemos la primera imagen del listado para ver que todo va bien
## plt.imshow(load_train_image(randurls[0])[0]+1/2)

In [0]:
# Cargamos los dataset con los listados de imágenes de entrenamiento y test

train_dataset = tf.data.Dataset.from_tensor_slices(tr_urls)
train_dataset = train_dataset.map(load_train_image, num_parallel_calls =tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(1)

# Visualización de lotes de imágenes
##for inimg, tgimg in train_dataset.take(5):
##  print(tgimg.shape)
##  plt.imshow(((tgimg[0,...])+1)/2)
##  plt.show()

test_dataset = tf.data.Dataset.from_tensor_slices(tr_urls)
test_dataset = train_dataset.map(load_test_image, num_parallel_calls =tf.data.experimental.AUTOTUNE)
test_dataset = train_dataset.batch(1)


In [0]:
# DISEÑO PIX2PIX

